In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
spark = SparkSession.builder \
    .config('spark.driver.memory', '3g') \
    .enableHiveSupport().getOrCreate()

In [2]:
! ls /data

Badges	Comments  PostLinks  Posts  Tags  Users  Votes


In [3]:
base_path = 'file:///data'

In [4]:
posts = spark.read.format('parquet').load(f'{base_path}/Posts/part-00500*')

In [5]:
posts.count()

71561

In [6]:
posts.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CommentCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)



In [7]:
posts.groupBy('PostTypeId').count().show()

+----------+-----+
|PostTypeId|count|
+----------+-----+
|         1|32773|
|         5|   44|
|         4|   44|
|         2|38700|
+----------+-----+



In [8]:
QUESTION_POST_TYPE = 1
questions = posts.where(col('PostTypeId') == QUESTION_POST_TYPE)

In [9]:
questions.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- ParentId: integer (nullable = true)
 |-- PostTypeId: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- Score: integer (nullable = true)
 |-- ViewCount: integer (nullable = true)
 |-- Body: string (nullable = true)
 |-- OwnerUserId: integer (nullable = true)
 |-- LastActivityDate: timestamp (nullable = true)
 |-- Title: string (nullable = true)
 |-- Tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- CommentCount: integer (nullable = true)
 |-- AnswerCount: integer (nullable = true)
 |-- LastEditDate: timestamp (nullable = true)
 |-- LastEditorUserId: integer (nullable = true)
 |-- AcceptedAnswerId: integer (nullable = true)
 |-- FavoriteCount: integer (nullable = true)
 |-- OwnerDisplayName: string (nullable = true)
 |-- ClosedDate: timestamp (nullable = true)
 |-- LastEditorDisplayName: string (nullable = true)
 |-- CommunityOwnedDate: timestamp (nullable = true)



### 1. Czy pytanie może być community-owned? (`CommunityOwnedDate`)

In [10]:
questions.where(col('CommunityOwnedDate').isNotNull()).select('Id').first()

### 2. Przykładowy ID zamkniętego pytania z września 2019 (`ClosedDate`)

https://stackoverflow.com/q/839899

In [11]:
questions.where(year('ClosedDate') == 2019) \
    .where(month('ClosedDate') == 9).select('Id').show()

+---+
| Id|
+---+
+---+



In [12]:

# mało badży (do tego czasu)
# mało tagów
# dużo tagów
# za krótkie pytanie -> mniej niż 100 znaków
# długość pytania
# długość tytułu
# wiek
# autor dopiero zarejestrowany (+1)
# niski view count
# tytuł kończy się znakiem zapytania
# tytuł krótszy niż 5 znaków
# CAPS lock w tytule
# dużo znaków interpunkcyjnych w tytule
# obecność obrazków
# "internet"
# "please"
# title zaczyna się wielką literą
# title zaczyna się od "how", "what" albo "why"
# godzina
# czy weekend?
# ile postów usera zostało zamkniętych (do tego czasu)
# ile pytań zadał dotąd
# ma "defaultowy" nick?
# wpisana lokacja?
# owner user id jest pusty?

## Cechy na początek:

 * ocena na minusie (+1)
 * obecność kodu (+1)
 * autor dopiero zarejestrowany (+1)

```
this is my code
``` 


-> <pre><code>


this is `my` code

--> <code>

In [13]:
questions \
    .show()

+--------+--------+----------+--------------------+-----+---------+--------------------+-----------+--------------------+--------------------+--------------------+------------+-----------+--------------------+----------------+----------------+-------------+----------------+--------------------+---------------------+------------------+
|      Id|ParentId|PostTypeId|        CreationDate|Score|ViewCount|                Body|OwnerUserId|    LastActivityDate|               Title|                Tags|CommentCount|AnswerCount|        LastEditDate|LastEditorUserId|AcceptedAnswerId|FavoriteCount|OwnerDisplayName|          ClosedDate|LastEditorDisplayName|CommunityOwnedDate|
+--------+--------+----------+--------------------+-----+---------+--------------------+-----------+--------------------+--------------------+--------------------+------------+-----------+--------------------+----------------+----------------+-------------+----------------+--------------------+---------------------+---------

In [14]:
import re
def how_many_code_letters(body):
    if body == None:
        return 0 ## or None?
    letters = 0
    for code in re.finditer('<code>([^<]+)</code>', body):
        letters += len(code.group(1))
    return letters

In [15]:
assert how_many_code_letters('<code>abc</code> <code>1</code>') == 4
assert how_many_code_letters('<pre><code>abc</code></pre> <code>1</code>') == 4
assert how_many_code_letters('''
<pre><code>line1
line2</code></pre> <code>1</code>
''') == 12

In [16]:
how_many_code_letters_udf = udf(how_many_code_letters, IntegerType())

In [17]:
users = spark.read.parquet(f'{base_path}/Users')
# users.printSchema()

In [18]:
days_difference_between_reg_and_post = datediff(questions.CreationDate, users.CreationDate)
input_df = questions.join(users, questions.OwnerUserId == users.Id) \
    .select(
        questions.Id,
        (col('Score')<0).alias('is_score_less_than_zero'),
        'Score',
        col('Body').contains('<code>').alias('has_code_attached'),
        how_many_code_letters_udf('Body').alias('code_length'),
        (days_difference_between_reg_and_post < 2).alias('is_author_just_registered'),
        days_difference_between_reg_and_post.alias('how_many_days_ago_user_registered'),
        col('ClosedDate').isNotNull().alias('is_closed'),
    )

## Przydatne funkcje

 * `unix_timestamp`
 * `regexp_extract`

In [19]:
input_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- is_score_less_than_zero: boolean (nullable = true)
 |-- Score: integer (nullable = true)
 |-- has_code_attached: boolean (nullable = true)
 |-- code_length: integer (nullable = true)
 |-- is_author_just_registered: boolean (nullable = true)
 |-- how_many_days_ago_user_registered: integer (nullable = true)
 |-- is_closed: boolean (nullable = false)



## MLLib

* `label` -> liczba
* `features` -> tablica liczb

In [20]:
from pyspark.ml.feature import 

In [21]:
assembler = VectorAssembler(inputCols=[
    'is_score_less_than_zero',
    'Score',
    'has_code_attached',
    'code_length',
    'is_author_just_registered',
    'how_many_days_ago_user_registered'
])

assembler.transform(input_df) \
    .withColumn('is_closed', col('is_closed').cast('int')) \
    .first()

Row(Id=53011229, is_score_less_than_zero=False, Score=14, has_code_attached=True, code_length=347, is_author_just_registered=False, how_many_days_ago_user_registered=3166, is_closed=0, VectorAssembler_41f9ce998f49__output=DenseVector([0.0, 14.0, 1.0, 347.0, 0.0, 3166.0]))